<h1> Activate chs_env virtual enviroment </h1>
Set-ExecutionPolicy -Scope CurrentUser -ExecutionPolicy Unrestricted <br>
chess_env\Scripts\activate

In [4]:
# Importing all relevant and necessary modules
import numpy as np 
import pandas as pd 
import os
import chess
import math
import matplotlib.pyplot as plt
import ollama
import random
from fentoboardimage import fenToImage, loadPiecesFolder

In [ ]:
def gen_legal_moves(board,frac=1):
    legal_moves = board.legal_moves
    legal_moves = [str(move) for move in legal_moves]
    if frac == 1:
        return legal_moves
    else:
        return np.random.choice(legal_moves, math.floor(frac*len(legal_moves)), replace=False)

def gen_prompt2(board,moves, n=10):
    return f"You are playing chess and the board state is {board}. You are playing as black and have to chose the best move to make next in order for black to win. Chose a move from this list of legal moves: {moves}, and please provide your next move in UCI notation?"
    
def gen_legal_moves(board,frac=1):
    legal_moves = board.legal_moves
    legal_moves = [str(move) for move in legal_moves]
    if frac == 1:
        return legal_moves
    else:
        return np.random.choice(legal_moves, math.floor(frac*len(legal_moves)), replace=False)

def prompt_move(prompt,board):
    response = ollama.chat(model='llama3', messages=[
            {'role': 'user','content': prompt,},])
    move = response['message']['content']
    for k in range(len(move)):
        if move[k:k+4] in gen_legal_moves(board):
            move = move[k:k+4]
            break
    return move

In [3]:
df = pd.read_csv(r"chessData.csv")

In [10]:
def prompt_ensemble(size,fen):
    moves = {}  #Dictionary to store the moves and their frequencies
    for i in range(size):
        board = chess.Board(fen)
        move = prompt_move(gen_prompt2(fen,gen_legal_moves(board)),board)
        if i % 1 == 0:
            print(f"Move {i}: {move}")
        if move in moves:
            moves[move] += 1 #Increment the frequency of the move
        else:
            moves[move] = 1 #Add the move to the dictionary
    return moves

# def ensemble_manager(df,sizes):
#     ensemble = {}
#     for size in sizes:
#         ensemble[size] = prompt_ensemble(size,df['FEN'][0]) #df['FEN'][0] is the first FEN in the dataset change for iteration over all FENs

#     return ensemble

In [11]:
fen = "rnbqkbnr/ppp1pppp/8/3p4/4P1Q1/8/PPPP1PPP/RNB1KBNR b KQkq - 1 2"
gen_legal_moves(chess.Board(fen))

['g8h6',
 'g8f6',
 'd8d7',
 'd8d6',
 'c8d7',
 'c8e6',
 'c8f5',
 'c8g4',
 'b8d7',
 'b8c6',
 'b8a6',
 'd5e4',
 'h7h6',
 'g7g6',
 'f7f6',
 'e7e6',
 'c7c6',
 'b7b6',
 'a7a6',
 'd5d4',
 'h7h5',
 'g7g5',
 'f7f5',
 'e7e5',
 'c7c5',
 'b7b5',
 'a7a5']

In [12]:
size = 25
ensemble = prompt_ensemble(size,fen)
ensemble

Move 0: d8d6
Move 1: d8d6
Move 2: d8d6
Move 3: d8d6
Move 4: d8d6
Move 5: d8d6
Move 6: d8d6
Move 7: d8d6
Move 8: d8d7


In [7]:
def show_board(fen_position,source = r"./pieces"):
    boardImage = fenToImage(
        fen = fen_position,
        squarelength = 100,
        pieceSet=loadPiecesFolder(source),
        darkColor="#D18B47",
        lightColor="#FFCE9E"
         )
    boardImage.show()
        
# fen = df['FEN'][321]
fen = "rnbqkbnr/ppp1pppp/8/3p4/4P1Q1/8/PPPP1PPP/RNB1KBNR b KQkq - 1 2"
board = chess.Board(fen)
# fen = board.fen()
show_board(fen)